# **Few shot adversarial learning of Realistic Neural Talking Head Models**
## Implementations, comparisons and discussion

When [MyHeritage.com](https://www.myheritage.com) announced the release of [Deep Nostalgia](https://www.myheritage.com/deep-nostalgia), a tool to animate faces from still photos, it received an overwhelming response from people. The website reported that over 1 million photos were animated in the first 48 hours alone and was reaching 3 million on the third day ([source](https://blog.myheritage.com/2021/02/deep-nostalgia-goes-viral/)). The world was going gaga over this and honestly, rightly so. It is only natural for a human to grab any chance they have at seeing their lost loved ones living again and when I found out about this tool while scrolling through my [reddit](https://www.reddit.com) feed, I also did not hesitate to jump on the bandwagon. Here is an example of a photo animated using Deep Nostalgia:

![example of Deep Nostalgia](https://media.giphy.com/media/vNEulvl9tMP2GDKpoz/giphy.gif) ![Another example](https://media.giphy.com/media/JRnVuIZKw5tPFPgdaH/giphy.gif)

I tried this tool on a photo of my grandfather who passed in the year 2003 and there he was, moving, looking around, smiling almost naturally. I showed my mother the animation on her birthday a few days later and I could see first-hand what the people meant in the reviews they gave this tool. Being a student of Artificial Intelligence, I couldn't help but wonder how this tool worked so I started looking for the tech behind it. It led me to the company that developed the technology behind it: [D-ID](https://www.d-id.com/). D-ID is a company which started with anonymization of facial featuress in photos using AI and then also developed [Talking Heads and the Live Portrait](https://www.d-id.com/reenactment/) which are a part of its reenactment suite.

Now, obviously the tech is proprietary so I could not find much details but I came across a [similar paper](https://arxiv.org/abs/1905.08233) from researchers at the Samsung AI center, Moscow. This blog post is going to be about this paper from the samsung AI lab: _Few-Shot Adversarial Learning of Realistic Neural Talking Head Models_. Feeling overwhelmed by the title of the paper? I was too. Let's change that.

# Overview

The researchers at the Samsing AI lab developed a [GAN (Generativce Adversarial Network)](https://arxiv.org/abs/1406.2661) to make realistic talking heads from photos of people with as few learning shots as possible. The model works by taking a source image and a target video/image and extracts the landmarks from the target image and applyting them to the source image, animating the source image into a talking head image. So, when applied, the implementation looks something like: 

Driver image:![driver image](https://media.giphy.com/media/5AXoAsfkw7ESOYzPH6/giphy.gif) Extracted landmarks:![landmarks gif](https://media.giphy.com/media/4V6KKfZT0OlOCWEN7z/giphy.gif)